# SEC EDGAR Financial Analytics

## Building a Compliance Search Tool with CocoIndex + Apache Doris

---

### The Problem

You're a fintech company building a **compliance search tool**. Your analysts need to quickly find relevant risk disclosures across thousands of SEC filings:

> *"Show me all cybersecurity risk mentions from tech companies in the last year, ranked by relevance."*

This sounds simple, but production requirements are complex:

| Requirement | Why It's Hard |
|-------------|---------------|
| **Semantic search** | "data breach" should match "unauthorized access to systems" |
| **Keyword precision** | When analysts search "GDPR", they want exact matches |
| **Temporal awareness** | 2024 filings matter more than 2019 filings |
| **Category filtering** | Filter by risk type (cyber, climate, regulatory) |
| **Multi-format data** | Filings (TXT), financial facts (JSON), exhibits (PDF) |
| **Audit trail** | Know exactly which document each result came from |
| **Incremental updates** | Daily SEC filings shouldn't require full reprocessing |

This tutorial shows you how to build it.

### What You'll Build

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        Compliance Search Tool                           │
├─────────────────────────────────────────────────────────────────────────┤
│  Query: "cybersecurity risks"                                           │
│  Filters: time_gate=365 days, topics=[RISK:CYBER], source=filing        │
│                                                                         │
│  Results:                                                               │
│  [0.032] Apple 10-K 2024 → "We face significant cybersecurity..."       │
│  [0.029] Microsoft 10-K 2024 → "Cyber threats continue to evolve..."    │
│  [0.025] JPMorgan 10-K 2024 → "We invest $700M in cybersecurity..."     │
└─────────────────────────────────────────────────────────────────────────┘
                                    ▲
                                    │ Hybrid Search
┌─────────────────────────────────────────────────────────────────────────┐
│                          Apache Doris                                   │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐                   │
│  │ HNSW Vector  │  │  Inverted    │  │   Array      │                   │
│  │    Index     │  │   Index      │  │  Columns     │                   │
│  │ (semantic)   │  │ (keywords)   │  │ (topics[])   │                   │
│  └──────────────┘  └──────────────┘  └──────────────┘                   │
└─────────────────────────────────────────────────────────────────────────┘
                                    ▲
                                    │ Incremental ETL
┌─────────────────────────────────────────────────────────────────────────┐
│                     CocoIndex Multi-Source Pipeline                     │
│                                                                         │
│  ┌──────────────┐   ┌──────────────┐   ┌──────────────┐                 │
│  │ TXT Filings  │   │ JSON Facts   │   │ PDF Exhibits │                 │
│  │ (10-K/10-Q)  │   │ (API Data)   │   │ (Documents)  │                 │
│  └──────┬───────┘   └──────┬───────┘   └──────┬───────┘                 │
│         │                  │                  │                         │
│         ▼                  ▼                  ▼                         │
│  ┌─────────────────────────────────────────────────────┐                │
│  │              Unified Chunk Collector                │                │
│  │   Scrub PII → Chunk → Embed → Extract Topics        │                │
│  └─────────────────────────────────────────────────────┘                │
│                                                                         │
│  ✓ Cached (only reprocess changed files)                                │
│  ✓ Incremental (daily updates in seconds)                               │
│  ✓ Traceable (full lineage from chunk to source)                        │
└─────────────────────────────────────────────────────────────────────────┘
```

---
## Part 1: Why Hybrid Search?

Let's understand why we need both vector and keyword search.

### The Semantic Gap

An analyst searches for **"cybersecurity risks"**:

```
Keyword search alone:
✗ MISSES: "We experienced unauthorized access to our systems..."
✗ MISSES: "Hackers exploited a vulnerability in our network..."
✓ FINDS:  "cybersecurity risks continue to threaten..."
```

Vector embeddings capture semantic meaning — "unauthorized access" is conceptually similar to "cybersecurity risks".

### The Precision Problem

But vector search alone has issues:

```
Query: "GDPR compliance"

Vector search returns:
✗ "We comply with various privacy regulations..." (too vague)
✗ "Data protection is important to us..." (no GDPR mention)
✓ "GDPR requires us to..." (what analyst wants)
```

When analysts use specific terms like "GDPR" or "SOX", they expect exact matches.

### The Solution: Hybrid Search with RRF

We combine both signals using **Reciprocal Rank Fusion (RRF)**:

```
rrf_score = 1/(k + semantic_rank) + 1/(k + lexical_rank)
```

- k=60 is the standard constant (no tuning needed)
- Used by Elasticsearch, Azure Cognitive Search
- Handles different score scales gracefully

---
## Part 2: Setup

### Prerequisites

- Python 3.11+
- Docker (for Apache Doris)
- ~5 minutes for setup

### Start the Infrastructure

```bash
# Start Doris and PostgreSQL (CocoIndex metadata store)
docker compose up -d

# Wait for Doris to be ready (~60 seconds)
docker compose logs -f doris-fe
# Look for: "Doris FE started successfully"
```

In [1]:
# Core imports
import os
from pathlib import Path
from datetime import timedelta

# CocoIndex - ETL framework
import cocoindex
import cocoindex.targets.doris as coco_doris

# Load environment
from dotenv import load_dotenv

load_dotenv()

print("Environment ready!")

Environment ready!


In [2]:
# Doris configuration (defaults work with docker-compose.yml)
DORIS_CONFIG = {
    "fe_host": os.environ.get("DORIS_FE_HOST", "localhost"),
    "fe_http_port": int(os.environ.get("DORIS_HTTP_PORT", "8030")),
    "query_port": int(os.environ.get("DORIS_QUERY_PORT", "9030")),
    "username": os.environ.get("DORIS_USERNAME", "root"),
    "password": os.environ.get("DORIS_PASSWORD", ""),
    "database": os.environ.get("DORIS_DATABASE", "sec_analytics"),
}

TABLE_CHUNKS = "filing_chunks"

print(f"Doris: {DORIS_CONFIG['fe_host']}:{DORIS_CONFIG['query_port']}")

Doris: localhost:9030


---
## Part 3: The Data

SEC provides data in **three different formats**, each serving different needs:

### 3.1 TXT Filings (10-K, 10-Q, 8-K)

Plain text annual/quarterly reports containing:
- **Item 1A: Risk Factors** — What could go wrong
- **Item 7: MD&A** — Management's analysis
- **Item 8: Financial Statements** — The numbers

```
data/filings/0000320193_2024-11-01_10-K.txt
             │          │          │
             │          │          └── Form type
             │          └───────────── Filing date
             └──────────────────────── CIK (company ID)
```

### 3.2 JSON Company Facts (SEC API)

Structured financial data from SEC's XBRL API:
- Revenue, net income, assets
- Time series data (multiple years)
- Machine-readable metrics

```
data/company_facts/0000320193.json
```

### 3.3 PDF Exhibits

Supporting documents attached to filings:
- Executive contracts
- Legal agreements
- Technical specifications

```
data/exhibits_pdf/0000320193_2024-ex21.pdf
```

**This tutorial shows how to ingest ALL THREE formats into a single searchable index.**

In [ ]:
# Create sample SEC data
from download import create_sample_data

create_sample_data()

---
## Part 4: The Transformation Pipeline

Before data becomes searchable, it passes through a series of transformations. Each step serves a specific purpose:

```
Raw Data → Metadata → Clean Text → Chunks → Embeddings + Topics → Index
```

### Transformation Overview

| Step | Function | Input → Output | Why It's Needed |
|------|----------|---------------|-----------------|
| **1. Metadata Extraction** | `extract_*_metadata()` | filename → struct | Enable filtering by CIK, date, form type |
| **2. Content Parsing** | `parse_company_facts()` | JSON → text | Make structured data searchable |
| **3. PII Scrubbing** | `scrub_pii()` | text → text | Remove emails, phones, SSNs |
| **4. Chunking** | `SplitRecursively()` | text → chunks[] | Create searchable units |
| **5. Embedding** | `text_to_embedding()` | text → vector | Enable semantic search |
| **6. Topic Tagging** | `extract_topics()` | text → topics[] | Enable category filtering |

### CocoIndex Function Patterns

Functions are defined in `functions.py` using the `@cocoindex.op.function` decorator:

```python
@cocoindex.op.function(cache=True, behavior_version=1)
def my_transform(input: str) -> Output:
    # cache=True: Results are cached, only recompute on input change
    # behavior_version: Bump this to invalidate cache when logic changes
    ...
```

This enables **incremental processing** — when you add new filings, only the new files are processed.

In [4]:
# Import all transformations from functions.py
# Keeping them in a separate file avoids "Function factory already exists" errors
# on notebook re-runs (Python caches the module, so decorators only execute once).
from functions import (
    # Data classes (return types)
    extract_filing_metadata,  # filename → FilingMetadata
    extract_json_metadata,  # (filename, content) → CompanyFactsMetadata
    parse_company_facts,  # JSON content → searchable text
    extract_pdf_metadata,  # filename → PdfMetadata
    scrub_pii,  # text → text (with PII removed)
    extract_topics,  # text → list[str] (topic tags)
    text_to_embedding,  # text → vector (384-dim)
)

print("Transformations loaded from functions.py")

Transformations loaded from functions.py


### Step 1: Metadata Extraction

Each data source encodes metadata in its filename. We extract this into structured fields for filtering:

```
0000320193_2024-11-01_10-K.txt
│          │          │
│          │          └── form_type: "10-K" (annual report)
│          └───────────── filing_date: "2024-11-01"
└──────────────────────── cik: "0000320193" (Apple's SEC ID)
```

**Why this matters**: Enables queries like "show 10-K filings from 2024" without scanning content.

In [5]:
# Test it
result = extract_filing_metadata("0000320193_2024-11-01_10-K.txt")
print(f"CIK:         {result.cik} (Apple Inc.)")
print(f"Filing Date: {result.filing_date}")
print(f"Form Type:   {result.form_type}")
print(f"Fiscal Year: {result.fiscal_year}")

CIK:         0000320193 (Apple Inc.)
Filing Date: 2024-11-01
Form Type:   10-K
Fiscal Year: 2024


### Step 2: Content Parsing (JSON → Searchable Text)

JSON company facts are structured data — great for machines, but not searchable by text queries.

**The problem**: A search for "Apple revenue" won't match `{"Revenues": {"val": 394328000000}}`.

**The solution**: `parse_company_facts()` converts JSON into natural language:

```
Input:  {"facts": {"us-gaap": {"Revenues": {"val": 394328000000, "end": "2024-09-28"}}}}
Output: "### Revenues\nRecent values: 2024-09-28: $394.3B"
```

Now the content can be chunked, embedded, and searched like any text document.

In [5]:
# Test JSON parsing
json_file = Path("data/company_facts/0000320193.json")
if json_file.exists():
    content = json_file.read_text()

    # Test metadata extraction
    meta = extract_json_metadata(json_file.name, content)
    print("Metadata:")
    print(f"  Entity: {meta.entity_name}")
    print(f"  CIK: {meta.cik}")
    print(f"  Latest Date: {meta.filing_date}")

    # Test content parsing
    print("\nParsed Content (first 800 chars):")
    print("-" * 50)
    parsed = parse_company_facts(content)
    print(parsed[:800] + "...")
else:
    print("No JSON file available yet - run create_sample_data() first")

Metadata:
  Entity: APPLE INC
  CIK: 0000320193
  Latest Date: 2024-11-01

Parsed Content (first 800 chars):
--------------------------------------------------
# Company Financial Facts: APPLE INC
CIK: 320193

## US-GAAP Financial Metrics

### Revenues
Amount of revenue recognized from goods sold, services rendered.
Recent values: 2024-09-28: $394.3B, 2023-09-30: $383.3B, 2022-09-24: $394.3B

### Net Income (Loss)
The portion of profit or loss for the period.
Recent values: 2024-09-28: $93.7B, 2023-09-30: $97.0B

### Assets
Sum of the carrying amounts as of the balance sheet date.
Recent values: 2024-09-28: $365.0B

### Research and Development Expense
The aggregate costs incurred for research and development.
Recent values: 2024-09-28: $29.9B
...


### Step 3: PDF Processing

PDFs require two transformations:

1. **Metadata extraction** (`extract_pdf_metadata`): Parse filename for CIK, date, exhibit type
2. **Content conversion** (`PdfToMarkdown`): Convert binary PDF to searchable markdown text

CocoIndex's built-in `PdfToMarkdown()` handles OCR, table extraction, and layout preservation.

In [6]:
# Test PDF metadata extraction
result = extract_pdf_metadata("0000320193_2024-ex21.pdf")
print(f"CIK:          {result.cik}")
print(f"Filing Date:  {result.filing_date}")
print(f"Exhibit Type: {result.exhibit_type}")
print(f"Form Type:    {result.form_type}")

CIK:          0000320193
Filing Date:  2024-ex21
Exhibit Type: EXHIBIT
Form Type:    EXHIBIT


### Step 4: PII Scrubbing

Remove personally identifiable information **before** chunking and indexing.

**Why scrub first?** Once text is chunked and embedded, PII becomes part of the searchable index. Scrubbing afterward requires reprocessing everything.

**Patterns removed**:
- SSN: `123-45-6789` → `[SSN REDACTED]`
- Phone: `(408) 996-1010` → `[PHONE REDACTED]`
- Email: `ir@apple.com` → `[EMAIL REDACTED]`

This runs early in the pipeline so PII never enters the vector index.

In [7]:
# Test it
test = "Contact IR at ir@apple.com or 408-996-1010"
print(f"Before: {test}")
print(f"After:  {scrub_pii(test)}")

Before: Contact IR at ir@apple.com or 408-996-1010
After:  Contact IR at [EMAIL REDACTED] or [PHONE REDACTED]


### Step 5: Topic Tagging

Extract structured categories from unstructured text. Each chunk gets a `topics[]` array:

```python
"We face cybersecurity risks..." → ["RISK:CYBER"]
"Climate change affects supply chain..." → ["RISK:CLIMATE", "RISK:SUPPLY"]
"Revenue grew 15%..." → ["TOPIC:FINANCIAL"]
```

**Why arrays?** Doris supports JSON array columns with efficient filtering using `json_contains()`:

```sql
-- Single topic filter
WHERE json_contains(topics, '"RISK:CYBER"')

-- Any matching topic (OR)
WHERE json_contains(topics, '"RISK:CYBER"') 
   OR json_contains(topics, '"RISK:CLIMATE"')

-- All matching topics (AND)
WHERE json_contains(topics, '"RISK:CYBER"') 
  AND json_contains(topics, '"RISK:REGULATORY"')
```

This enables **faceted search** — filter by category, then rank by relevance.

In [8]:
# Test it
samples = [
    "We face significant cybersecurity risks including ransomware.",
    "Climate change poses risks to our supply chain operations.",
    "Our AI investments include machine learning infrastructure.",
    "Revenue increased 15% to $394.3 billion with net income of $97 billion.",
]

for text in samples:
    print(f"Text: {text[:50]}...")
    print(f"Topics: {extract_topics(text)}\n")

Text: We face significant cybersecurity risks including ...
Topics: ['RISK:CYBER', 'RISK:REGULATORY']

Text: Climate change poses risks to our supply chain ope...
Topics: ['RISK:CLIMATE', 'RISK:SUPPLY']

Text: Our AI investments include machine learning infras...
Topics: ['TOPIC:AI']

Text: Revenue increased 15% to $394.3 billion with net i...
Topics: ['TOPIC:FINANCIAL']



### Step 6: Text Embedding

Convert text chunks to 384-dimensional vectors using `sentence-transformers/all-MiniLM-L6-v2`.

**Why embeddings matter**: They capture semantic meaning, not just keywords.

```
Query: "unauthorized access to systems"
  ↓ embedding
Similar to: "cybersecurity breach", "data leak", "hacking incident"
```

The `@cocoindex.transform_flow` decorator ensures:
- **Same model** for indexing and querying (critical for accuracy)
- **Efficient batching** during indexing
- **Async support** for query-time embedding

---
## Part 5: The Multi-Source ETL Pipeline

Now we assemble everything into a complete pipeline that ingests **all three data formats**.

### Pipeline Structure

```
┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
│  TXT Filings    │  │  JSON Facts     │  │  PDF Exhibits   │
│  data/filings/  │  │  data/company_  │  │  data/exhibits_ │
│  *.txt          │  │  facts/*.json   │  │  pdf/*.pdf      │
└────────┬────────┘  └────────┬────────┘  └────────┬────────┘
         │                    │                    │
         ▼                    ▼                    ▼
┌──────────────────────────────────────────────────────────────┐
│                   Unified Chunk Collector                     │
│                                                              │
│  Each source:                                                │
│    1. Extracts metadata (filename parsing)                   │
│    2. Parses content (text/JSON → searchable text, PDF → md) │
│    3. Scrubs PII                                             │
│    4. Chunks text                                            │
│    5. Embeds + extracts topics                               │
│    6. Collects with source_type field                        │
└────────────────────────────┬─────────────────────────────────┘
                             │
                             ▼
┌──────────────────────────────────────────────────────────────┐
│                      Apache Doris                            │
│  • HNSW vector index (semantic search)                       │
│  • Inverted index (keyword search)                           │
│  • source_type field for filtering by format                 │
└──────────────────────────────────────────────────────────────┘
```

**Key pattern: Multiple sources → Single collector**

In [9]:
def process_and_collect(doc, text_field: str, metadata, collector):
    """
    Common chunk processing for all source types.

    This helper encapsulates the shared ETL pattern:
    1. Scrub PII from text
    2. Split into chunks (1000 chars, 200 overlap)
    3. Generate embeddings
    4. Extract topic tags
    5. Collect into unified index

    Args:
        doc: The document row from a source
        text_field: Name of the field containing text to process
        metadata: Extracted metadata (includes cik, filing_date, form_type,
                  fiscal_year, source_type)
        collector: The chunk collector to add results to
    """
    # Scrub PII before any processing
    doc["scrubbed"] = doc[text_field].transform(scrub_pii)

    # Split into searchable chunks
    doc["chunks"] = doc["scrubbed"].transform(
        cocoindex.functions.SplitRecursively(),
        language="markdown",
        chunk_size=1000,
        chunk_overlap=200,
    )

    # Process each chunk: embed + tag + collect
    with doc["chunks"].row() as chunk:
        chunk["embedding"] = text_to_embedding(chunk["text"])
        chunk["topics"] = chunk["text"].transform(extract_topics)

        collector.collect(
            chunk_id=cocoindex.GeneratedField.UUID,
            source_type=metadata["source_type"],
            doc_filename=doc["filename"],
            location=chunk["location"],
            cik=metadata["cik"],
            filing_date=metadata["filing_date"],
            form_type=metadata["form_type"],
            fiscal_year=metadata["fiscal_year"],
            text=chunk["text"],
            embedding=chunk["embedding"],
            topics=chunk["topics"],
        )


print("Helper function defined: process_and_collect()")

Helper function defined: process_and_collect()


In [10]:
# Close existing flow if re-running this cell (safe no-op on first run)
try:
    sec_filing_flow.close()  # type: ignore[name-defined]
except NameError:
    pass  # First run — no flow to close

# Check if PdfToMarkdown is available (requires optional dependency)
_has_pdf = hasattr(cocoindex.functions, "PdfToMarkdown")
if not _has_pdf:
    print("Note: PdfToMarkdown not available — PDF exhibits will be skipped.")


@cocoindex.flow_def(name="SECFilingAnalytics")
def sec_filing_flow(
    flow_builder: cocoindex.FlowBuilder, data_scope: cocoindex.DataScope
) -> None:
    """
    SEC Filing Analytics Pipeline - Multi-Source

    Ingests TXT, JSON, and optionally PDF into a unified searchable index.
    """

    # ═══════════════════════════════════════════════════════════════════
    # SOURCES
    # ═══════════════════════════════════════════════════════════════════
    data_scope["txt_filings"] = flow_builder.add_source(
        cocoindex.sources.LocalFile(path="data/filings", included_patterns=["*.txt"]),
        refresh_interval=timedelta(hours=1),
    )
    data_scope["json_facts"] = flow_builder.add_source(
        cocoindex.sources.LocalFile(
            path="data/company_facts", included_patterns=["*.json"]
        ),
        refresh_interval=timedelta(hours=1),
    )

    # ═══════════════════════════════════════════════════════════════════
    # UNIFIED COLLECTOR - all sources feed into this
    # ═══════════════════════════════════════════════════════════════════
    chunk_collector = data_scope.add_collector()

    # ═══════════════════════════════════════════════════════════════════
    # PROCESS EACH SOURCE
    # ═══════════════════════════════════════════════════════════════════

    # TXT Filings: extract metadata from filename, use content directly
    with data_scope["txt_filings"].row() as filing:
        filing["metadata"] = filing["filename"].transform(extract_filing_metadata)
        process_and_collect(filing, "content", filing["metadata"], chunk_collector)

    # JSON Facts: extract metadata (needs both filename and content),
    # then parse JSON to searchable text
    with data_scope["json_facts"].row() as facts:
        # Multi-input: filename is implicit first arg, content is kwarg
        facts["metadata"] = facts["filename"].transform(
            extract_json_metadata, content=facts["content"]
        )
        facts["parsed"] = facts["content"].transform(parse_company_facts)
        process_and_collect(facts, "parsed", facts["metadata"], chunk_collector)

    # PDF Exhibits: convert PDF to markdown (requires PdfToMarkdown)
    if _has_pdf:
        data_scope["pdf_exhibits"] = flow_builder.add_source(
            cocoindex.sources.LocalFile(
                path="data/exhibits_pdf", included_patterns=["*.pdf"], binary=True
            ),
            refresh_interval=timedelta(hours=1),
        )
        with data_scope["pdf_exhibits"].row() as pdf:
            pdf["metadata"] = pdf["filename"].transform(extract_pdf_metadata)
            pdf["markdown"] = pdf["content"].transform(
                cocoindex.functions.PdfToMarkdown()
            )
            process_and_collect(pdf, "markdown", pdf["metadata"], chunk_collector)

    # ═══════════════════════════════════════════════════════════════════
    # EXPORT TO DORIS
    # ═══════════════════════════════════════════════════════════════════
    chunk_collector.export(
        "filing_chunks",
        coco_doris.DorisTarget(
            fe_host=DORIS_CONFIG["fe_host"],
            fe_http_port=DORIS_CONFIG["fe_http_port"],
            query_port=DORIS_CONFIG["query_port"],
            username=DORIS_CONFIG["username"],
            password=DORIS_CONFIG["password"],
            database=DORIS_CONFIG["database"],
            table=TABLE_CHUNKS,
        ),
        primary_key_fields=["chunk_id"],
        vector_indexes=[
            cocoindex.VectorIndexDef(
                field_name="embedding",
                metric=cocoindex.VectorSimilarityMetric.L2_DISTANCE,
            )
        ],
        fts_indexes=[
            cocoindex.FtsIndexDef(field_name="text", parameters={"parser": "unicode"})
        ],
    )


print("Pipeline defined: SECFilingAnalytics")

Note: PdfToMarkdown not available — PDF exhibits will be skipped.
Pipeline defined: SECFilingAnalytics


In [11]:
# Initialize CocoIndex
cocoindex.init()
print("CocoIndex initialized")

CocoIndex initialized


In [14]:
# Setup and run the pipeline
print("Setting up database tables...")
await sec_filing_flow.setup_async()

print("\nProcessing SEC data from all sources...")
print("  - TXT filings (10-K/10-Q)")
print("  - JSON company facts")
if _has_pdf:
    print("  - PDF exhibits")
print("\n(First run loads the embedding model)\n")

await sec_filing_flow.update_async()

print("\nDone! Data is now searchable in Doris.")

Setting up database tables...
2026-02-04T20:05:03.908582Z  INFO setup.apply_changes_for_flow_ctx{flow_name=SECFilingAnalytics}:setup.apply_changes_for_flow{flow_name=SECFilingAnalytics}: cocoindex_engine::execution::db_tracking_setup: Cleaning up tracking metadata and target data for 1 stale source(s): [6]
2026-02-04T20:05:03.916399Z  INFO setup.apply_changes_for_flow_ctx{flow_name=SECFilingAnalytics}:setup.apply_changes_for_flow{flow_name=SECFilingAnalytics}: cocoindex_engine::execution::db_tracking_setup: Processed 0 tracking entries, deleted 0 target rows
2026-02-04T20:05:03.917932Z  INFO setup.apply_changes_for_flow_ctx{flow_name=SECFilingAnalytics}:setup.apply_changes_for_flow{flow_name=SECFilingAnalytics}: cocoindex_engine::execution::db_tracking_setup: Deleted 0 tracking entries for stale sources

Processing SEC data from all sources...
  - TXT filings (10-K/10-Q)
  - JSON company facts

(First run loads the embedding model)


Done! Data is now searchable in Doris.


---
## Part 6: Search Modes

Now the fun part — searching our indexed filings! We provide **four search modes**, each optimized for different use cases:

| Mode | Function | Best For | Example Query |
|------|----------|----------|---------------|
| **Hybrid** | `search()` | General queries | "cybersecurity risks" |
| **Lexical** | `search_lexical()` | Exact terms, acronyms | "GDPR", "SOX compliance" |
| **Topic** | `search_by_topics()` | Category filtering | "risks" + filter by RISK:CYBER |
| **Portfolio** | `search_portfolio()` | Cross-company comparison | Compare Apple vs Microsoft |

### How They Differ

```
Query: "data protection requirements"

Hybrid search:
  ✓ Finds "GDPR compliance obligations" (semantic match)
  ✓ Finds "data protection requirements" (keyword match)
  → Best overall results via RRF fusion

Lexical search:
  ✓ Finds "data protection requirements" (exact keywords)
  ✗ Misses "GDPR compliance obligations" (no keyword overlap)
  → Use when you need precise term matching

Topic search:
  ✓ Pre-filters to chunks tagged RISK:REGULATORY
  ✓ Then ranks by semantic similarity
  → Use when you know the category

Portfolio search:
  ✓ Returns top-K results PER company
  → Use for cross-company analysis
```

In [15]:
# =============================================================================
# SEARCH HELPERS (from search.py)
# =============================================================================
# Low-level SQL utilities are in search.py to keep the notebook focused
# on the search logic itself.

import importlib
import search as search_mod

importlib.reload(search_mod)

from search import (
    extract_keywords,
    format_embedding,
    format_list,
    build_where,
    doris_query,
)

# Common table reference
TABLE = f"{DORIS_CONFIG['database']}.{TABLE_CHUNKS}"

print("SQL helpers loaded from search.py")

SQL helpers loaded from search.py


### Mode 1: Hybrid Search (Recommended Default)

**When to use**: General-purpose queries where you want both semantic understanding AND keyword precision.

**How it works**: Reciprocal Rank Fusion (RRF) combines two rankings:
1. **Semantic rank**: How similar is the meaning? (vector distance)
2. **Lexical rank**: Does it contain the keywords? (inverted index)

```
RRF Score = 1/(60 + semantic_rank) + 1/(60 + lexical_rank)
```

Documents that rank high in BOTH signals get boosted to the top.

```python
# Basic hybrid search
await search("cybersecurity risks")

# With time filter (last 365 days)
await search("cybersecurity risks", time_gate_days=365)

# With source filter (only filings, not JSON facts)
await search("cybersecurity risks", source_types=["filing", "exhibit"])
```

In [16]:
async def search(
    query: str,
    time_gate_days: int | None = None,
    source_types: list[str] | None = None,
    limit: int = 10,
) -> list[dict]:
    """
    Hybrid search using RRF (Reciprocal Rank Fusion).

    Combines semantic similarity and keyword matching.
    """
    # Prepare query components
    embedding = format_embedding(await text_to_embedding.eval_async(query))
    keywords = extract_keywords(query)

    # Build filter conditions
    conditions = []
    if time_gate_days:
        conditions.append(
            f"filing_date >= DATE_SUB(CURRENT_DATE(), INTERVAL {time_gate_days} DAY)"
        )
    if source_types:
        conditions.append(f"source_type IN ({format_list(source_types)})")
    where = build_where(conditions)

    # RRF hybrid search
    sql = f"""
    WITH
    semantic AS (
        SELECT chunk_id, doc_filename, cik, filing_date, source_type, text, topics,
               ROW_NUMBER() OVER (ORDER BY l2_distance(embedding, {embedding})) AS rank
        FROM {TABLE} WHERE {where}
    ),
    lexical AS (
        SELECT chunk_id,
               ROW_NUMBER() OVER (ORDER BY CASE WHEN text MATCH_ANY '{keywords}' THEN 0 ELSE 1 END) AS rank
        FROM {TABLE} WHERE {where}
    )
    SELECT s.*, l.rank AS lex_rank,
           1.0/(60 + s.rank) + 1.0/(60 + l.rank) AS score
    FROM semantic s JOIN lexical l USING (chunk_id)
    ORDER BY score DESC LIMIT {limit}
    """

    return [
        {
            "doc_filename": r[1],
            "cik": r[2],
            "filing_date": str(r[3]) if r[3] else None,
            "source_type": r[4],
            "text": r[5],
            "topics": r[6] or [],
            "sem_rank": r[7],
            "lex_rank": r[8],
            "rrf_score": float(r[9]),
        }
        for r in await doris_query(DORIS_CONFIG, sql)
    ]


print("Hybrid search ready!")

Hybrid search ready!


---
### Mode 2: Lexical Search

**When to use**: Exact terms, acronyms, regulatory codes, or when you need precise keyword matching.

```python
# Find exact mentions of "supply chain" as a phrase
await search_lexical("supply chain", match_type="phrase")

# Find chunks with ANY of these terms (OR)
await search_lexical("supply chain", match_type="any")

# Find chunks with ALL of these terms (AND)
await search_lexical("supply chain", match_type="all")
```

**Doris Inverted Index Operators**:

| Operator | SQL | Behavior | Strictness |
|----------|-----|----------|------------|
| `MATCH_ANY` | `text MATCH_ANY 'supply chain'` | supply OR chain | Broadest |
| `MATCH_ALL` | `text MATCH_ALL 'supply chain'` | supply AND chain | Stricter |
| `MATCH_PHRASE` | `text MATCH_PHRASE 'supply chain'` | Exact phrase | Strictest |

**Relevance Ranking**: Doris provides `SCORE()` — a BM25-based relevance score computed during full-text matching. We use `ORDER BY SCORE() DESC` to return the most relevant results first.

In [17]:
async def search_lexical(
    query: str,
    match_type: str = "any",  # "any", "all", or "phrase"
    source_types: list[str] | None = None,
    limit: int = 10,
) -> list[dict]:
    """
    Keyword search using Doris inverted index.

    match_type: "any" (OR), "all" (AND), "phrase" (exact sequence)

    Results are ranked by BM25 relevance using Doris SCORE().
    """
    keywords = extract_keywords(query)
    match_op = {"any": "MATCH_ANY", "all": "MATCH_ALL", "phrase": "MATCH_PHRASE"}[
        match_type
    ]

    # Build conditions
    conditions = [f"text {match_op} '{keywords}'"]
    if source_types:
        conditions.append(f"source_type IN ({format_list(source_types)})")

    sql = f"""
    SELECT doc_filename, cik, filing_date, source_type, text, topics, SCORE() AS score
    FROM {TABLE}
    WHERE {build_where(conditions)}
    ORDER BY score DESC
    LIMIT {limit}
    """

    return [
        {
            "doc_filename": r[0],
            "cik": r[1],
            "filing_date": str(r[2]) if r[2] else None,
            "source_type": r[3],
            "text": r[4],
            "topics": r[5] or [],
            "score": float(r[6]),
        }
        for r in await doris_query(DORIS_CONFIG, sql)
    ]


print("Lexical search ready!")

Lexical search ready!


In [18]:
# Compare MATCH_ANY (OR) vs MATCH_ALL (AND) vs MATCH_PHRASE (exact)
# Using "supply chain" — a phrase that demonstrates all three operators well.

print("=" * 70)
print("MATCH_ANY (OR): Finds chunks with 'supply' OR 'chain'")
print("-" * 70)
results = await search_lexical("supply chain", match_type="any", limit=3)
for i, r in enumerate(results, 1):
    print(f"{i}. [score={r['score']:.1f}] {r['text'][:80]}...")

print("\n" + "=" * 70)
print("MATCH_ALL (AND): Finds chunks with BOTH 'supply' AND 'chain'")
print("-" * 70)
results = await search_lexical("supply chain", match_type="all", limit=3)
for i, r in enumerate(results, 1):
    print(f"{i}. [score={r['score']:.1f}] {r['text'][:80]}...")

print("\n" + "=" * 70)
print("MATCH_PHRASE (exact): Finds chunks with exact phrase 'supply chain'")
print("-" * 70)
results = await search_lexical("supply chain", match_type="phrase", limit=3)
for i, r in enumerate(results, 1):
    print(f"{i}. [score={r['score']:.1f}] {r['text'][:80]}...")

print("\nResults ranked by BM25 score via Doris SCORE().")
print("Notice: ANY (broadest) → ALL (stricter) → PHRASE (strictest).")

MATCH_ANY (OR): Finds chunks with 'supply' OR 'chain'
----------------------------------------------------------------------
1. [score=8.7] APPLE INC. FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CYBERSECURITY RISKS
Th...
2. [score=8.3] The Company has a large, global business with sales outside the U.S. representin...
3. [score=7.9] Major public health issues, including pandemics such as the COVID-19 pandemic, h...

MATCH_ALL (AND): Finds chunks with BOTH 'supply' AND 'chain'
----------------------------------------------------------------------
1. [score=8.7] APPLE INC. FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CYBERSECURITY RISKS
Th...
2. [score=8.3] The Company has a large, global business with sales outside the U.S. representin...
3. [score=7.9] Major public health issues, including pandemics such as the COVID-19 pandemic, h...

MATCH_PHRASE (exact): Finds chunks with exact phrase 'supply chain'
----------------------------------------------------------------------
1. [scor

#### Example: Hybrid Search with RRF Scoring

Notice how RRF combines semantic rank and lexical rank. Documents scoring well on BOTH get boosted.

In [19]:
# Hybrid search: semantic + keyword combined via RRF
results = await search(
    "cybersecurity risks and data breach", time_gate_days=365, limit=5
)

print("Query: 'cybersecurity risks and data breach'")
print("=" * 75)
print(
    f"{'#':<3} {'RRF Score':<12} {'Sem Rank':<10} {'Lex Rank':<10} {'Source':<10} File"
)
print("-" * 75)

for i, r in enumerate(results, 1):
    print(
        f"{i:<3} {r['rrf_score']:.4f}       #{r['sem_rank']:<8} #{r['lex_rank']:<8} {r['source_type']:<10} {r['doc_filename'][:30]}"
    )

print("\n" + "=" * 75)
print(
    "How to read: Lower rank = better. RRF boosts docs that rank well on BOTH signals."
)
print("\nTop result preview:")
if results:
    r = results[0]
    print(f"  Topics: {r['topics']}")
    print(f"  Text: {r['text'][:200]}...")

Query: 'cybersecurity risks and data breach'
#   RRF Score    Sem Rank   Lex Rank   Source     File
---------------------------------------------------------------------------
1   0.0328       #1        #1        filing     0000789019_2025-07-30_10-K.txt
2   0.0313       #6        #2        filing     0000320193_2025-10-31_10-K.txt
3   0.0310       #4        #5        filing     0000320193_2025-10-31_10-K.txt
4   0.0298       #2        #13       filing     0000320193_2025-10-31_10-K.txt
5   0.0295       #5        #11       filing     0000789019_2025-07-30_10-K.txt

How to read: Lower rank = better. RRF boosts docs that rank well on BOTH signals.

Top result preview:
  Topics: ["RISK:CYBER","RISK:REGULATORY"]
  Text: Business
3
Information about our Executive Officers
14
Item 1A.
Risk Factors
16
Item 1B.
Unresolved Staff Comments
30
Item 1C.
Cybersecurity
30
Item 2.
Properties
32
Item 3.
Legal Proceedings
32
Item ...


#### Example: Filter by Source Type

Search only specific data formats using `source_types` parameter.

In [20]:
# Search ONLY in filings (after multi-source indexing, you can also filter by "facts" or "exhibit")
print("Source filter: filing only")
print("=" * 60)
results = await search("revenue net income assets", source_types=["filing"], limit=3)
for i, r in enumerate(results, 1):
    print(f"{i}. [{r['source_type']}] {r['doc_filename']}")
    print(f"   {r['text'][:100]}...\n")

Source filter: filing only
1. [filing] 0000320193_2025-10-31_10-K.txt
   The Company has historically experienced higher net sales in its first quarter compared to other qua...

2. [filing] 0000320193_2025-10-31_10-K.txt
   Markets and Distribution
The Company’s customers are primarily in the consumer, small and mid-sized ...

3. [filing] 0000789019_2025-07-30_10-K.txt
   Indicate by check mark whether the registrant has filed a report on and attestation to its managemen...



In [21]:
# Search filings + exhibits (skip JSON facts)
print("Source filter: filing, exhibit (skip JSON)")
print("=" * 60)
results = await search("risk disclosure", source_types=["filing", "exhibit"], limit=3)
for i, r in enumerate(results, 1):
    print(f"{i}. [{r['source_type']}] {r['doc_filename'][:40]}")
    print(f"   Topics: {r['topics']}\n")

Source filter: filing, exhibit (skip JSON)
1. [filing] 0000789019_2025-07-30_10-K.txt
   Topics: ["RISK:CYBER","RISK:REGULATORY"]

2. [filing] 0000789019_2025-07-30_10-K.txt
   Topics: ["RISK:REGULATORY"]

3. [filing] 0000789019_2025-07-30_10-K.txt
   Topics: ["RISK:REGULATORY"]



---
### Mode 3: Topic-Filtered Search

**When to use**: You know the category/risk type and want to search within that subset.

**How it works**: 
1. First, filter chunks by topic tags (using `json_contains()`)
2. Then, rank by semantic similarity within that subset

**Available Topics** (extracted by `extract_topics()` in `functions.py`):
```
RISK:CYBER      - Cybersecurity, data breaches, ransomware
RISK:CLIMATE    - Climate change, emissions, sustainability  
RISK:SUPPLY     - Supply chain, logistics, shortages
RISK:REGULATORY - Compliance, SEC, regulations
TOPIC:AI        - Artificial intelligence, machine learning
TOPIC:CLOUD     - Cloud computing, AWS, Azure, SaaS
TOPIC:FINANCIAL - Revenue, income, assets, cash flow
```

```python
# Find AI-related content (semantic search within TOPIC:AI chunks)
await search_by_topics("company strategy", topics=["TOPIC:AI"])

# Find content with ANY of these topics (OR)
await search_by_topics("company risks", topics=["RISK:CYBER", "RISK:CLIMATE"], match="any")

# Find content with ALL of these topics (AND) - more restrictive
await search_by_topics("regulatory cyber", topics=["RISK:CYBER", "RISK:REGULATORY"], match="all")
```

In [22]:
async def search_by_topics(
    query: str,
    topics: list[str],
    match: str = "any",  # "any" or "all"
    source_types: list[str] | None = None,
    limit: int = 10,
) -> list[dict]:
    """
    Semantic search filtered by topic tags.

    match="any": Has ANY of these topics (OR)
    match="all": Has ALL of these topics (AND)

    Note: topics column is stored as JSON in Doris, so we use json_contains()
    instead of array_contains() for filtering.
    """
    embedding = format_embedding(await text_to_embedding.eval_async(query))

    # Build topic filter using json_contains (topics is JSON type in Doris)
    if match == "any":
        # OR: any of the topics
        topic_conditions = [f"json_contains(topics, '\"{t}\"')" for t in topics]
        conditions = ["(" + " OR ".join(topic_conditions) + ")"]
    else:
        # AND: all of the topics
        conditions = [f"json_contains(topics, '\"{t}\"')" for t in topics]

    if source_types:
        conditions.append(f"source_type IN ({format_list(source_types)})")

    sql = f"""
    SELECT doc_filename, cik, filing_date, source_type, text, topics,
           l2_distance(embedding, {embedding}) AS score
    FROM {TABLE}
    WHERE {build_where(conditions)}
    ORDER BY score LIMIT {limit}
    """

    return [
        {
            "doc_filename": r[0],
            "cik": r[1],
            "filing_date": str(r[2]) if r[2] else None,
            "source_type": r[3],
            "text": r[4],
            "topics": r[5] or [],
            "score": float(r[6]),
        }
        for r in await doris_query(DORIS_CONFIG, sql)
    ]


print("Topic search ready!")

Topic search ready!


In [23]:
# Topic filter: Only chunks tagged with TOPIC:AI
print("Topic filter: TOPIC:AI")
print("=" * 60)
results = await search_by_topics(
    "company strategy investments", topics=["TOPIC:AI"], limit=3
)

for i, r in enumerate(results, 1):
    print(f"{i}. [{r['source_type']}] {r['doc_filename']}")
    print(f"   Score: {r['score']:.3f} | Topics: {r['topics']}")
    print(f"   {r['text'][:80]}...\n")

Topic filter: TOPIC:AI
1. [filing] 0000320193_2024-11-01_10-K.txt
   Score: 1.217 | Topics: ["RISK:CYBER","RISK:CLIMATE","RISK:SUPPLY","RISK:REGULATORY","TOPIC:AI"]
   APPLE INC. FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CYBERSECURITY RISKS
Th...

2. [filing] 0000789019_2025-07-30_10-K.txt
   Score: 1.232 | Topics: ["RISK:REGULATORY","TOPIC:AI","TOPIC:CLOUD"]
   PART
I
ITEM 1. B
USINESS
GENERAL
Microsoft is a technology company committed to ...

3. [filing] 0000789019_2024-10-15_10-K.txt
   Score: 1.289 | Topics: ["RISK:CYBER","RISK:REGULATORY","TOPIC:AI","TOPIC:CLOUD"]
   MICROSOFT CORPORATION FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CLOUD INFRA...



---
### Mode 4: Portfolio Search

**When to use**: Compare how multiple companies discuss the same topic.

**How it works**:
1. Filter to specified companies (by CIK)
2. Rank by semantic similarity
3. Use `PARTITION BY cik` to get top-K results PER company

This ensures each company is represented in results, even if one company has more relevant content.

```python
# Compare cybersecurity across 3 companies (top 2 chunks each)
await search_portfolio(
    "cybersecurity investments",
    ciks=["0000320193", "0000789019", "0000019617"],  # Apple, Microsoft, JPMorgan
    top_k=2
)

# Compare only within filings (skip JSON facts and PDF exhibits)
await search_portfolio(
    "AI strategy",
    ciks=["0000320193", "0000789019"],
    source_types=["filing"],
    top_k=3
)
```

**Common CIKs** (for reference):
```
0000320193  Apple
0000789019  Microsoft
0000019617  JPMorgan Chase
0001018724  Amazon
0001652044  Alphabet (Google)
0001326801  Meta (Facebook)
```

In [24]:
async def search_portfolio(
    query: str,
    ciks: list[str],
    source_types: list[str] | None = None,
    top_k: int = 2,
) -> list[dict]:
    """
    Get top results per company for cross-company comparison.

    Uses PARTITION BY to rank within each company.
    """
    embedding = format_embedding(await text_to_embedding.eval_async(query))

    conditions = [f"cik IN ({format_list(ciks)})"]
    if source_types:
        conditions.append(f"source_type IN ({format_list(source_types)})")

    sql = f"""
    WITH ranked AS (
        SELECT cik, doc_filename, source_type, text, topics,
               l2_distance(embedding, {embedding}) AS score,
               ROW_NUMBER() OVER (PARTITION BY cik ORDER BY l2_distance(embedding, {embedding})) AS rank
        FROM {TABLE}
        WHERE {build_where(conditions)}
    )
    SELECT cik, doc_filename, source_type, text, topics, score
    FROM ranked WHERE rank <= {top_k}
    ORDER BY cik, score
    """

    return [
        {
            "cik": r[0],
            "doc_filename": r[1],
            "source_type": r[2],
            "text": r[3],
            "topics": r[4] or [],
            "score": float(r[5]),
        }
        for r in await doris_query(DORIS_CONFIG, sql)
    ]


print("Portfolio search ready!")

Portfolio search ready!


In [25]:
# Portfolio comparison: Top result per company
companies = {"0000320193": "Apple", "0000789019": "Microsoft", "0000019617": "JPMorgan"}

print("Portfolio Comparison: 'cybersecurity investments'")
print("=" * 65)
results = await search_portfolio(
    "cybersecurity investments", ciks=list(companies.keys()), top_k=1
)

for r in results:
    name = companies.get(r["cik"], r["cik"])
    print(f"\n{name} ({r['cik']}) [{r['source_type']}]")
    print(f"  Score: {r['score']:.3f} | Topics: {r['topics']}")
    print(f"  {r['text'][:120]}...")

Portfolio Comparison: 'cybersecurity investments'

JPMorgan (0000019617) [filing]
  Score: 1.072 | Topics: ["RISK:CYBER","RISK:CLIMATE","RISK:REGULATORY"]
  JPMORGAN CHASE & CO. FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CREDIT RISK
The Company faces significant credit ris...

Apple (0000320193) [filing]
  Score: 1.018 | Topics: ["RISK:CYBER","RISK:CLIMATE","RISK:SUPPLY","RISK:REGULATORY","TOPIC:AI"]
  APPLE INC. FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CYBERSECURITY RISKS
The Company faces significant cybersecurit...

Microsoft (0000789019) [filing]
  Score: 1.060 | Topics: ["RISK:CYBER","RISK:REGULATORY","TOPIC:AI","TOPIC:CLOUD"]
  MICROSOFT CORPORATION FORM 10-K ANNUAL REPORT
ITEM 1A. RISK FACTORS

CLOUD INFRASTRUCTURE
Our Azure cloud platform faces...


---
## Part 7: Access Control

Apache Doris provides **row-level** and **column-level** security for compliance requirements.

### Column-Level Security

Hide sensitive columns from certain users (e.g., analysts shouldn't see raw embeddings):

```sql
-- Create analyst role with limited column access
GRANT SELECT_PRIV (
    chunk_id, doc_filename, cik, filing_date, text, topics
    -- embedding column NOT included
) ON sec_analytics.filing_chunks TO 'analyst'@'%';
```

### Row-Level Security

Restrict users to specific companies or data subsets:

```sql
-- Tech analyst can only see Apple and Microsoft filings
CREATE ROW POLICY tech_analyst_policy ON filing_chunks
AS RESTRICTIVE TO 'tech_analyst'@'%'
USING (cik IN ('0000320193', '0000789019'));

-- Compliance officer can only see regulatory-tagged content
-- Note: topics is JSON type, so use json_contains()
CREATE ROW POLICY compliance_policy ON filing_chunks
AS RESTRICTIVE TO 'compliance'@'%'
USING (json_contains(topics, '"RISK:REGULATORY"'));

-- Time-based restriction (only filings from 2024+)
CREATE ROW POLICY recent_only_policy ON filing_chunks
AS RESTRICTIVE TO 'junior_analyst'@'%'
USING (filing_date >= '2024-01-01');
```

### Source Type Restrictions

Limit access to specific data formats:

```sql
-- User can only search 10-K filings, not PDF exhibits
CREATE ROW POLICY filing_only_policy ON filing_chunks
AS RESTRICTIVE TO 'filing_viewer'@'%'
USING (source_type = 'filing');
```

**Key point**: These policies are enforced at the database level — all queries (including our search functions) automatically respect them.

---
## Summary

You've built a production-ready SEC filing search system with:

| Feature | Implementation |
|---------|----------------|
| **Multi-format ingestion** | TXT (filings) + JSON (API) + PDF (exhibits) |
| **Unified search** | Single collector, `source_type` field for filtering |
| **Semantic search** | SentenceTransformer embeddings + HNSW index |
| **Keyword search** | Doris inverted index (MATCH_ANY/ALL/PHRASE) |
| **Hybrid ranking** | RRF fusion (no weight tuning needed) |
| **Temporal awareness** | Time gating via WHERE clause |
| **Category filtering** | topics[] JSON array with `json_contains()` |
| **Portfolio analysis** | `PARTITION BY` for per-company results |
| **Access control** | Row/column-level security via `CREATE ROW POLICY` |
| **Incremental updates** | CocoIndex caching |
| **Audit trail** | doc_filename + location + source_type lineage |

### Search API

```python
# Hybrid search (semantic + keyword)
await search("cybersecurity risks", time_gate_days=365)

# Lexical search (keyword only)
await search_lexical("GDPR", match_type="phrase")

# Topic-filtered search
await search_by_topics("risks", topics=["RISK:CYBER"])

# Portfolio comparison (top-k per company)
await search_portfolio("cybersecurity", ciks=["0000320193", "0000789019"])
```

### Project Files

| File | Purpose |
|------|---------|
| `functions.py` | CocoIndex transformation functions (ETL) |
| `search.py` | Low-level Doris SQL helpers |
| `download.py` | Sample data creation |

### Next Steps

1. **View lineage**: `cocoindex server -ci main.py`
2. **Add more data**: Extend `download.py` with additional companies

### Resources

- [CocoIndex Documentation](https://cocoindex.io/docs)
- [Apache Doris Vector Search](https://doris.apache.org/docs)
- [SEC EDGAR API](https://www.sec.gov/developer)